### Урок 3. Системы управления базами данных MongoDB и SQLite в Python

1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.
3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [1]:
# pip install pymongo

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pymongo import MongoClient

In [3]:
class Job():
    
    def __init__(self, db_url, db_name, collection_name):
        self.client = MongoClient(db_url)
        self.db = self.client[db_name]
        self.collection = self.db[collection_name]

    def add_to_db(self, vacancy):
        if self.collection.find_one({'link': {'$eq': vacancy['link']}}):
            self.collection.update_one({'link': vacancy['link']}, {'$set': vacancy})
        else:
            self.collection.insert_one(vacancy)
            
    def search_by_salary(self, salary):
        result = self.collection.find({'$or': [{'salary_min': {'$gt': salary}}, {'salary_max': {'$gt': salary}}]})
        return result
    
    def get_html(self, url, params):
        response = requests.get(url, params=params).text
        html = bs(response, 'html.parser')

        return html

    def get_last_page(self, html):
        pagination = html.find('a', {'class': 'f-test-button-1'})
        if not pagination:
            last_page = 1
        else:
            pagination = pagination.findParent()
            last_page = int(pagination.find_all('a')[-2].text)

        return last_page

    def get_vacancy_details(self, vacancy):
        vacancy_details = {'city': [],
                           'name': [],
                           'company': [],
                           'salary_min': [],
                           'salary_max': [],
                           'link': [], 
                           'site': []
                          }
        # city
        vacancy_details['city'] = vacancy.find(attrs={'class': 'f-test-text-company-item-location'})\
        .findChildren()[2].text.split(',')[0]

        # job title
        vacancy_details['name'] = vacancy.find('a', href=True).text

        # job link
        vacancy_details['link'] = 'https://www.superjob.ru' + vacancy.find('a', href=True)['href']

        # company name
        vacancy_details['company'] = vacancy.find(attrs={'class': 'f-test-text-vacancy-item-company-name'}).text if \
            vacancy.find(attrs={'class': 'f-test-text-vacancy-item-company-name'}) else None

        # job site
        vacancy_details['site'] = 'Superjob'

        # salary
        salary = vacancy.find(attrs={'class': 'f-test-text-company-item-salary'}).next.text
        salary = re.split(r'\s|-', salary)
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1]+salary[2])
        elif salary[0] == 'от':
            salary_min = int(salary[1]+salary[2])
            salary_max = None
        elif salary[0] == 'По':
            salary_min = None
            salary_max = None
        else:
            salary_min = int(salary[0]+salary[1])
            salary_max = int(salary[3]+salary[4]) if len(salary) > 3 else salary_min
        vacancy_details['salary_min'] = salary_min
        vacancy_details['salary_max'] = salary_max

        return vacancy_details

    def get_vacancies(self, jobname):
        url = 'https://russia.superjob.ru/vacancy/search/'
        params = {
            'keywords': jobname,
            'profession_only': 1,
            'page': ''
        }
        html = self.get_html(url, params)
        for page in range(1, self.get_last_page(html) + 1):
            params['page'] = page
            html = self.get_html(url, params)
            if html.find_all(attrs={'class': 'f-test-vacancy-item'}):
                results = html.find_all(attrs={'class': 'f-test-vacancy-item'})
            for result in results:
                result = self.get_vacancy_details(result)
                self.add_to_db(result)

In [4]:
parser = Job('localhost:27017','newdb','vacancies')
parser.get_vacancies('Data')

In [5]:
df = pd.DataFrame(parser.search_by_salary(100000))
df

,_id,city,name,company,salary_min,salary_max,link,site
0,6165ac7ec6060c5a960d9036,Екатеринбург,Data science инженер,Сима-ленд,200000.0,250000.0,https://www.superjob.ru/vakansii/data-science-...,Superjob
1,6165ac7ec6060c5a960d9038,Москва,Senior Python Developer / Data Engineer,Национальный исследовательский университет Выс...,NaN,300000.0,https://www.superjob.ru/vakansii/senior-python...,Superjob
2,6165ac7ec6060c5a960d9039,Москва,Middle Data Scientist,Автоматизация и консалтинг,104000.0,NaN,https://www.superjob.ru/vakansii/middle-data-s...,Superjob
3,6165ac7ec6060c5a960d9044,Санкт-Петербург,Data Analyst (Analytics Team) / Аналитик баз д...,Semrush,160000.0,NaN,https://www.superjob.ru/vakansii/data-analyst-...,Superjob
4,6165ac80c6060c5a960d9045,Санкт-Петербург,Lead data analyst / Product owner (Analytics T...,Semrush,370000.0,460000.0,https://www.superjob.ru/vakansii/lead-data-ana...,Superjob
5,6165ac80c6060c5a960d9046,Москва,Senior Data Scientist,Автоматизация и консалтинг,174000.0,NaN,https://www.superjob.ru/vakansii/senior-data-s...,Superjob
6,6165ac80c6060c5a960d9047,Москва,Senior Data analyst / Техлид отдела анализа да...,Автоматизация и консалтинг,174000.0,NaN,https://www.superjob.ru/vakansii/senior-data-a...,Superjob
7,6165ac80c6060c5a960d9048,Москва,Middle Data analyst / Ведущий аналитик данных,Автоматизация и консалтинг,104000.0,NaN,https://www.superjob.ru/vakansii/middle-data-a...,Superjob
8,6165ac80c6060c5a960d9049,Москва,Middle Data Engineer / Python разработчик,Автоматизация и консалтинг,104000.0,NaN,https://www.superjob.ru/vakansii/middle-data-e...,Superjob


![dbscreen](./dbscreen.png)